<a href="https://colab.research.google.com/github/mgonzalz/ppd_titanic-modelos/blob/main/titanic_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Estudio del Titanic: Clasificador binario.**
Partimos del dataset `titanic_train.csv` que contiene datos de 891 pasajeros del Titanic.

Las columnas son las siguientes:

*   `PassengerId`: número del pasajero.
*   `Survived`: si sobrevivió.
*   `Pclass`: clase en la que viajaba.
*   `Name`: nombre del pasajero.
*   `Sex`: hombre o mujer.
*   `Age`: edad.
*   `SibSp`: número de hermanos o cónyuges con los que viajaba.
*   `Parch`: número de padres o hijos con los que viajaba.
*   `Ticket`: nº del billete.
*   `Fare`: importe del billete.
*   `Cabin`: camarote en el que viajaba.
*   `Embarked`: puerto en el que embarcó.

Se parangonarán todos los modelos de clasificación vistos hasta ahora, evaluando su precisión y determinando cuál es el enfoque más efectivo para analizar los datos.

## Importación e instalación.
Para evitar la subida de archivos manualmente en Google Colab, clonaremos directamente el repositorio donde se encuentran alojados los archivos necesarios para la evaluación del modelo y luego los añadiremos al directorio actual.

In [ ]:
!git clone https://github.com/mgonzalz/ppd_titanic-modelos.git

Cloning into 'ppd_titanic-modelos'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8 (delta 0), reused 5 (delta 0), pack-reused 0
Receiving objects: 100% (8/8), 22.69 KiB | 505.00 KiB/s, done.


In [ ]:
!mv ppd_titanic-modelos/* ./ # Mover los archivos al directorio general

mv: cannot move 'ppd_titanic-modelos/data' to './data': Directory not empty


In [ ]:
!rm -rf ppd_titanic-modelos # Eliminación de la carpeta vacía

El archivo `requirements.txt` contiene todas las versiones de los paquetes utilizados. A través de la siguiente secuencia de código los instalaremos e importaremos los métodos necesarios.

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Python estándar
import os
import sys
from typing import List

# Configuración de Spark
import findspark
import pyspark
from pyspark.sql import DataFrame, SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

# Modelado y evaluación de Spark ML
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, \
                                      GBTClassifier, RandomForestClassifier, LinearSVC, NaiveBayes, \
                                      MultilayerPerceptronClassifier

## Creación del entorno.
Se preparará el entorno para el uso de Spark, descargando este y configurando las variables de entorno necesarias. Mediante `findspark` se habilitará la accesibilidad de Spark desde el entorno de ejecución. Finalmente, se crea la sesión de Spark.

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

findspark.init()
findspark.find()

spark= SparkSession \
       .builder \
       .appName("Clasificador-Binario") \
       .getOrCreate()

spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
46 packages can be upgraded. Run 'apt list --upgradable' to see them.
tar: spark-3.2.1-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recov

In [ ]:
spark=SparkSession.builder.appName('Clasificador-Binario').getOrCreate()

## Preparación del dataset.
Antes de la realización de modelos de clasificación, debemos de preparar los datos y tener un conocimiento sobre estos.

In [ ]:
data = spark.read.csv("/content/data/titanic_train (1).csv", header= True, inferSchema=True, sep =',')
data.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

Analicemos este dataset. A primera vista vemos que para el estudio podemos prescindir del `PassengerId` ya que se trata del ID del pasajero y no aporta información para el estudio. Pasa lo mismo con la columna `Name`, el nombre del pasajero no influye en el estudio, y `Ticket`. Por ello, serán suprimidas.

In [ ]:
data = data.drop('PassengerId', 'Name', 'Ticket')
data.show(5)

+--------+------+------+----+-----+-----+-------+-----+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Cabin|Embarked|
+--------+------+------+----+-----+-----+-------+-----+--------+
|       0|     3|  male|22.0|    1|    0|   7.25| NULL|       S|
|       1|     1|female|38.0|    1|    0|71.2833|  C85|       C|
|       1|     3|female|26.0|    0|    0|  7.925| NULL|       S|
|       1|     1|female|35.0|    1|    0|   53.1| C123|       S|
|       0|     3|  male|35.0|    0|    0|   8.05| NULL|       S|
+--------+------+------+----+-----+-----+-------+-----+--------+
only showing top 5 rows



In [ ]:
print(f"Total de datos: {data.count()}")

Total de datos: 891


### Valores nulos.
La posesión de valores nulos afecta al estudio. Se puede presuponer que al eliminar todos estos, el dataset esta completamente limpio. Sin embargo, la supresión por completo de ellos puede ocasionar un error en el estudio, por ello es necesario su estudio y análisis.

In [ ]:
data.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
null = data.toPandas().isnull().sum() # Valores nulos.
null

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [ ]:
print("\033[1mColumnas de datos nulos:\033[0m")
for col, count in null.items():
   if count > 0:
    print('   La columna', col, 'tiene aproximadamente un', round(count/data.count()*100, 3), '% de datos nulos.')

Columnas de datos nulos:
   La columna Age tiene aproximadamente un 19.865 % de datos nulos.
   La columna Cabin tiene aproximadamente un 77.104 % de datos nulos.
   La columna Embarked tiene aproximadamente un 0.224 % de datos nulos.


Cuándo se tienen columnas con un alto número de valores nulos, lo mejor es la eliminación de estas columnas ya que si se borran las filas, el dataset se quedaría con pocos datos.

Observamos que la columna `Cabin` tiene mas del 70% de datos nulos. Esta columna informa del camarote en el que viaja el tripulante. Lo mejor para esta columna será la eliminación de ella.

En el caso de la columna `Embarked` al ser solo 2 filas con valores nulos, se eliminarán estas ya que su eliminación no afectará al estudio. Ya se albergan suficientes datos.

Sin embargo, la columna `Age` tiene menos del 20% de datos nulos. Lo que realizaremos será la sustitución de los datos faltantes utilizando la **mediana**. La mediana es una buena medida de estimación para sustituir ya que no se ve afectada por valores atípicos, a diferencia de la media.



In [ ]:
data = data.drop('Cabin')

In [ ]:
data = data.dropna(subset=["Embarked"]) #eliminación filas nulas

In [ ]:
median = data.agg(F.median('Age')).collect()[0]
data = data.fillna(median[0], subset=["Age"])
data.show(5)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 5 rows



### Valores duplicados.
En la celda posterior observamos que no hay filas duplicadas. En caso de haberlas, tendríamos que eliminarlas.

In [ ]:
data.subtract(data.dropDuplicates()).show()

+--------+------+---+---+-----+-----+----+--------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Embarked|
+--------+------+---+---+-----+-----+----+--------+
+--------+------+---+---+-----+-----+----+--------+



In [ ]:
print(f"Total de datos: {data.count()}")

Total de datos: 889


### Label Encoding: Transformación de variables categóricas a tipo numérico.
Primero haremos un estudio de las variables categóricas, y miraremos el número de datos únicos que hay y dependiendo del número que haya se hará la codificación ordinal o *Label Encoding* manualmente o automaticamente mediante `StringIndexer`.

In [ ]:
data.show(3)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 3 rows



In [ ]:
data.dtypes

[('Survived', 'int'),
 ('Pclass', 'int'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Fare', 'double'),
 ('Embarked', 'string')]

In [ ]:
print('\033[1mValores únicos:\033[0m')
for i in data.dtypes:
    if i[1] == 'string':
        distinct_count = data.select(i[0]).distinct().count()
        print(f"   Número de valores únicos en la columna '{i[0]}':", distinct_count)

Valores únicos:
   Número de valores únicos en la columna 'Sex': 2
   Número de valores únicos en la columna 'Embarked': 3


#### **Sex: Codificación ordinal manual.**
En este caso solo hay 2 valores únicos y vamos a decir lo siguiente:
*   Por cada instancia de `male` (hombre), asignaremos el valor **1**.
*    Por cada instancia de `female` (mujer), asignaremos el valor **0**.



In [ ]:
data.groupBy('Sex').count().show() #otra opción sin que muestre la cuenta es con la opción .distinct()

+------+-----+
|   Sex|count|
+------+-----+
|female|  312|
|  male|  577|
+------+-----+



In [ ]:
data = data.withColumn('Sex', F.when(data.Sex == "male", 1).otherwise(0))

#### **Embarked: Codificación ordinal automática, StringIndexer.**
Vemos que hay tres valores únicos, a cada uno de ellos se le asignará un número: **0, 1, 2**.

In [ ]:
data.select('Embarked').distinct().show()

+--------+
|Embarked|
+--------+
|       Q|
|       C|
|       S|
+--------+



In [ ]:
indexer_model = StringIndexer(inputCol="Embarked", outputCol="Embarked_StringIndexer").fit(data)
data = indexer_model.transform(data)

In [ ]:
print("\033[1mAsignación numérica:\033[0m")
for index, value in enumerate(indexer_model.labels):
  print(f'   Por cada instancia de {value}, asignaremos el valor {index}.')

Asignación numérica:
   Por cada instancia de S, asignaremos el valor 0.
   Por cada instancia de C, asignaremos el valor 1.
   Por cada instancia de Q, asignaremos el valor 2.


In [ ]:
# Copia de la nueva columna en la anterior y eliminación de la nueva.
data = data.withColumn("Embarked", data["Embarked_StringIndexer"])
data = data.drop("Embarked_StringIndexer")
data.show(5)

+--------+------+---+----+-----+-----+-------+--------+
|Survived|Pclass|Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+---+----+-----+-----+-------+--------+
|       0|     3|  1|22.0|    1|    0|   7.25|     0.0|
|       1|     1|  0|38.0|    1|    0|71.2833|     1.0|
|       1|     3|  0|26.0|    0|    0|  7.925|     0.0|
|       1|     1|  0|35.0|    1|    0|   53.1|     0.0|
|       0|     3|  1|35.0|    0|    0|   8.05|     0.0|
+--------+------+---+----+-----+-----+-------+--------+
only showing top 5 rows



Con todo lo realizado, podemos considerar ya el dataset limpio y preparado para su posterior examinación. En caso de necesitarlo, realizaremos una copia.

In [ ]:
df_filter = data.select(*data.columns)

### Estudio general de los datos.
Antes de la realización de los modelos haremos un análisis genérico para la comprensión de qué características estuvieron asociadas con la supervivencia de las personas en el Titanic.

In [ ]:
data.describe().show()

+-------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-----------------+-------------------+
|summary|           Survived|            Pclass|                Sex|               Age|             SibSp|              Parch|             Fare|           Embarked|
+-------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-----------------+-------------------+
|  count|                889|               889|                889|               889|               889|                889|              889|                889|
|   mean|0.38245219347581555|2.3115860517435323| 0.6490438695163104|29.315151856017994|0.5241844769403825|0.38245219347581555|32.09668087739029|0.36220472440944884|
| stddev|0.48625968831477334|0.8346997785705753|0.47753789445536743|12.984932293690775| 1.103704875596923| 0.8067607445174785|49.69750431670795| 0.6361572404817025|
|    min| 

En cuanto a la **media** concluimos que alrededor del 38% de los pasajeros sobrevivieron. Su desviación estándar sugiere una alta variabilidad de los datos.


In [ ]:
data.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  340|
|       0|  549|
+--------+-----+



Se observa una disparidad significativa en la cantidad de datos entre las dos clases de la variable objetivo. Para abordar este desequilibrio y evitar que el modelo se sesgue hacia la clase mayoritaria, se deberia de realizar un `OverSampling` o  **sobremuestreo**. Esto implica generar datos para la clase minoritaria de forma aleatoria basándose en los datos pasados,aumentando así su representación y equilibrando la distribución de clases.

#### **Matriz de correlación.**
La matriz de correlación permite mostrar las relaciones de todas las variables del dataframe entre ellas. Una buena relación se considera cercana a +-1. Si comparamos según nuestra variable dependiente `Survived` vemos que el género de la persona influyó en si sobrevive o no por ser superior al 50%.

In [ ]:
data.toPandas().corr(method='pearson')

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,1.000000,-0.335549,-0.541585,-0.069822,-0.034040,0.083151,0.255290,0.108669
Pclass,-0.335549,1.000000,0.127741,-0.336512,0.081656,0.016824,-0.548193,0.043835
Sex,-0.541585,0.127741,1.000000,0.086506,-0.116348,-0.247508,-0.179958,-0.118593
Age,-0.069822,-0.336512,0.086506,1.000000,-0.232543,-0.171485,0.093707,-0.007165
SibSp,-0.034040,0.081656,-0.116348,-0.232543,1.000000,0.414542,0.160887,-0.060606
Parch,0.083151,0.016824,-0.247508,-0.171485,0.414542,1.000000,0.217532,-0.079320
Fare,0.255290,-0.548193,-0.179958,0.093707,0.160887,0.217532,1.000000,0.063462
Embarked,0.108669,0.043835,-0.118593,-0.007165,-0.060606,-0.079320,0.063462,1.000000


In [ ]:
data.groupBy("Sex").count().show()

+---+-----+
|Sex|count|
+---+-----+
|  1|  577|
|  0|  312|
+---+-----+



In [ ]:
data.groupBy("Sex").agg({"Survived": "mean"}).show()

+---+-------------------+
|Sex|      avg(Survived)|
+---+-------------------+
|  1|0.18890814558058924|
|  0| 0.7403846153846154|
+---+-------------------+



La diferencia de medias sugiere que hubo una tendencia mucho mayor para que las mujeres (`female == 0`) sobrevivieran en comparación con los hombres.

Si se compara con su correlación sacamos al misma conclusión, ya que esta es negativa. El ser negativa indica una lrelación lineal negativa, es decir, que existe una tendencia a que las personas con un valor más bajo tengan una mayor probabilidad de supervivencia en comparación con aquellas con un valor más alto.

In [ ]:
data.groupBy("SibSp").agg({"Survived": "mean"}).orderBy(data["SibSp"].asc()).show() #orden ascendente

+-----+-------------------+
|SibSp|      avg(Survived)|
+-----+-------------------+
|    0| 0.3432343234323432|
|    1| 0.5358851674641149|
|    2| 0.4642857142857143|
|    3|               0.25|
|    4|0.16666666666666666|
|    5|                0.0|
|    8|                0.0|
+-----+-------------------+



La columna `SibSp` muestra el número de hermanos/cónyuges a bordo. A partir de la matriz de correlación vemos que no hay una relación lineal fuerte (no que no exista) entre esta y `Survived`. Si analizamos la tabla anterior obtenemos que tienen tendencia a sobrevivir las personas que como máximo tienen a bordo 2 cónyuges y hermanos.

In [ ]:
data.groupBy("Parch").agg({"Survived": "mean"}).orderBy(data["Parch"].asc()).show()

+-----+-------------------+
|Parch|      avg(Survived)|
+-----+-------------------+
|    0|0.34171597633136097|
|    1| 0.5508474576271186|
|    2|                0.5|
|    3|                0.6|
|    4|                0.0|
|    5|                0.2|
|    6|                0.0|
+-----+-------------------+



En `Parch` se muestra el número de padres/hijos a bordo. A partir de la tabla vemos que tienen una tasa de superviviencia más alta las personas con 1,2 o 3 padres e hijos a bordo.

Esto sugiere que el número de padres/hijos a bordo puede haber influido en las posibilidades de supervivencia, con tasas de supervivencia más altas para aquellos que tenían un número moderado de compañeros de viaje.

Sin embargo la correlación de `Parch` respecto a la variable dependiente es cercana a cero, indicando que no hay una relación lineal fuerte (no que no exista) entre estas.

## Realización de los modelos de clasificación binaria.
A continuación para no repetir el código, pondremos los datos que tienen en común todas ellas. La variable dependiente es
```Survived```.



In [ ]:
depent_cols = 'Survived'

In [ ]:
feature_cols = [col for col in data.columns if col != depent_cols] # Variables independientes.

Primero para no modificar el dataset, lo copiaremos y modificaremos la copia.

In [ ]:
df = data.select(*data.columns)

In [ ]:
assembler = VectorAssembler(inputCols=feature_cols,
                            outputCol="features")
df = assembler.transform(df)

Mediante el `MinMaxScaler` transformamos las características escalándolas a un rango dado, por defecto (0,1). Este se utiliza para mejorar el modelo en caso de tener funciones con rangos muy diferentes y querer escalarlas a un rango común.

In [ ]:
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
pipeline = Pipeline(stages=[scaler])
scalerModel = pipeline.fit(df)
scaledData = scalerModel.transform(df)

Hemos dividido los datos en un 80% para el entrenamiento y un 20% para el test.`Seed` permite establecer una semilla para la generación de números aleatorios, lo que asegura que cada vez que se ejecute este código, la división sea consistente. Este valor va desde 0 hasta 100.

In [ ]:
train_data, test_data = scaledData.randomSplit([0.8, 0.2], seed=61)

### Árbol de decisión.
El árbol de decisión se trata de un modelo de clasificación binaria que representa decisiones y sus resultados en forma de gráfico de tipo árbol. Cada nodo representa una pregunta, cada rama una posible respuesta, y cada hoja una conclusión.

Los inputs necesarios para la realización de `DecisionTreeClassifier` son:
  - `labelCol`: De tipo *double*, es decir, decimal.
  - `featuresCol`: Este debe de ser un vector con las variables independientes.

In [ ]:
dt = DecisionTreeClassifier(labelCol=depent_cols,
                            featuresCol="scaled_features")
dtModel = dt.fit(train_data)

La función `featureImportances` devuelve un vector disperso con la siguiente información: primero el número de variables independientes utilizadas; entre llaves se trata de un diccionario dónde la clave es el índice de una característica y el valor indica la importancia de esa característica en el modelo. A partir de ello, se puede saber cuánto contribuye cada característica a las decisiones de clasificación.

In [ ]:
dtModel.featureImportances

SparseVector(7, {0: 0.1467, 1: 0.5875, 2: 0.0847, 3: 0.0602, 4: 0.0206, 5: 0.1003})

In [ ]:
print(dtModel)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_4b0218608f1b, depth=5, numNodes=29, numClasses=2, numFeatures=7


Su predicción devuelve tres nuevas columnas:
  - `rawPrediction`: De tipo vector, tiene la longitud del número de clases de la variable dependiente. Cada valor en el vector cuenta cuántas veces aparece cada categoría entre los datos de entrenamiento que llegaron a ese punto del árbol.
  - `probability`: De tipo vector, tiene la longitud del número de clases de la variable dependiente. Normalizado a una distribución multinomial.
  - `prediction`: De tipo *double*, es decir, decimal. Columna predictora. Predice la columna dependiente. Del vector de probabilidad, el que mayor tenga es la predicción.

In [ ]:
predictions = dtModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol=depent_cols, predictionCol="prediction", metricName="accuracy")
accuracy_dt = evaluator.evaluate(predictions)

In [ ]:
predictions.show(7)

+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+-------------+--------------------+----------+
|Survived|Pclass|Sex| Age|SibSp|Parch|    Fare|Embarked|            features|     scaled_features|rawPrediction|         probability|prediction|
+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+-------------+--------------------+----------+
|       0|     1|  1|27.0|    0|    2|   211.5|     1.0|[1.0,1.0,27.0,0.0...|[0.0,1.0,0.334003...|   [27.0,4.0]|[0.87096774193548...|       0.0|
|       0|     1|  1|28.0|    0|    0| 27.7208|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|  [48.0,29.0]|[0.62337662337662...|       0.0|
|       0|     1|  1|28.0|    0|    0| 30.6958|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|  [48.0,29.0]|[0.62337662337662...|       0.0|
|       0|     1|  1|28.0|    0|    0|    42.4|     0.0|[1.0,1.0,28.0,0.0...|(7,[1,2,5],[1.0,0...|  [48.0,29.0]|[0.62337662337662.

**Recuento de las predicciones**

En lo siguiente concluimos lo siguiente:
  - Cuando predecimos que sobrevive, 1, y verdaderamente lo hace. El modelo ha acertado un total de 42 veces. Sin embargo, el modelo ha fallado un total de 21 veces.
  - Cuando predecimos que no sobrevive, 0, y verdaderamente no lo hace. El modelo ha acertado un total de 114 veces. Sin embargo, el modelo ha fallado un total de 6 veces.

Observamos que predice bastante bien si no sobrevive. Pero la predicción de si sobrevive es baja debido a los pocos datos que se tienen.

In [ ]:
predictions.select("prediction", depent_cols).groupBy("prediction", depent_cols).count().show()

+----------+--------+-----+
|prediction|Survived|count|
+----------+--------+-----+
|       1.0|       0|    6|
|       0.0|       0|  114|
|       0.0|       1|   21|
|       1.0|       1|   42|
+----------+--------+-----+



In [ ]:
print(f'Con el modelo de arbol de decisión hemos obtenido una precisión de {accuracy_dt:.4f}')

Con el modelo de arbol de decisión hemos obtenido una precisión de 0.8525


### Árbol de decisión: Boosting del gradiente.
La diferencia de este con el modelo de árbol de decisión es que se cobinan múltiples árboles de decisión secuencialmente. Cada nuevo árbol corrige el error del anterior.
Los parámetros indicados son los siguientes:
  - `labelCol`: De tipo *double*, es decir, decimal.
  - `featuresCol`: Este debe de ser un vector con las variables independientes.
  - `maxIter`: De tipo *int*, el número máximo de árboles.

Al igual que en el modelo de Árbol de decisión, la predicción devuelve las mismas columnas.


In [ ]:
gbt = GBTClassifier(labelCol=depent_cols,
                    featuresCol="scaled_features",
                    maxIter=20)
gbtModel = gbt.fit(train_data)

Observamos que el árbol cosntruido contiene 20 árboles, indicados en `maxIter`; un total de dos clases, correspondientes a la variable dependiente; un total de siete variables independientes, `feature_cols`.

In [ ]:
print(gbtModel)

GBTClassificationModel: uid = GBTClassifier_22f22559f33d, numTrees=20, numClasses=2, numFeatures=7


In [ ]:
predictions = gbtModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol=depent_cols, predictionCol="prediction", metricName="accuracy")
accuracy_gbt = evaluator.evaluate(predictions)

In [ ]:
predictions.show(7)

+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex| Age|SibSp|Parch|    Fare|Embarked|            features|     scaled_features|       rawPrediction|         probability|prediction|
+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|       0|     1|  1|27.0|    0|    2|   211.5|     1.0|[1.0,1.0,27.0,0.0...|[0.0,1.0,0.334003...|[0.11166180988048...|[0.55560001645553...|       0.0|
|       0|     1|  1|28.0|    0|    0| 27.7208|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[0.06686365664247...|[0.53338209549020...|       0.0|
|       0|     1|  1|28.0|    0|    0| 30.6958|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[0.06686365664247...|[0.53338209549020...|       0.0|
|       0|     1|  1|28.0|    0|    0|    42.4|     0.0|[1.0,1.0,28.0,0.0...|(7,[1,2,5],

**Recuento de las predicciones**

En lo siguiente concluimos lo siguiente:
  - Cuando predecimos que sobrevive, 1, y verdaderamente lo hace. El modelo ha acertado un total de 44 veces. Sin embargo, el modelo ha fallado un total de 19 veces.
  - Cuando predecimos que no sobrevive, 0, y verdaderamente no lo hace. El modelo ha acertado un total de 115 veces. Sin embargo, el modelo ha fallado un total de 5 veces.

In [ ]:
predictions.select("prediction", depent_cols).groupBy("prediction", depent_cols).count().show()

+----------+--------+-----+
|prediction|Survived|count|
+----------+--------+-----+
|       1.0|       0|    5|
|       0.0|       0|  115|
|       0.0|       1|   19|
|       1.0|       1|   44|
+----------+--------+-----+



In [ ]:
print(f'Con el modelo de arbol de decisión con boosting del gradiente hemos obtenido una precisión de {accuracy_gbt:.4f}')

Con el modelo de arbol de decisión con boosting del gradiente hemos obtenido una precisión de 0.8689


### **Regresión Logística.**
La regresión logística se trata de un método estadístico que trata de modelar la probabilidad de una variable cualitativa binaria.
Los inputs necesarios para la realización de `DecisionTreeClassifier` son:
  - `labelCol`: De tipo *double*, es decir, decimal.
  - `featuresCol`: Este debe de ser un vector con las variables independientes.

Al igual que en el modelo de Árbol de decisión, la predicción devuelve las mismas columnas.

In [ ]:
lr = LogisticRegression(featuresCol="features", labelCol=depent_cols)

lrModel = lr.fit(train_data) #ajuste del modelo

In [ ]:
predictions = lrModel.transform(test_data)

multi_evaluator = MulticlassClassificationEvaluator(labelCol=depent_cols, predictionCol="prediction")
accuracy_lr = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "accuracy"})

In [ ]:
predictions.select("prediction", depent_cols).groupBy("prediction", depent_cols).count().show()

+----------+--------+-----+
|prediction|Survived|count|
+----------+--------+-----+
|       1.0|       0|   15|
|       0.0|       0|  105|
|       0.0|       1|   19|
|       1.0|       1|   44|
+----------+--------+-----+



**Recuento de las predicciones**

En lo siguiente concluimos lo siguiente:
  - Cuando predecimos que sobrevive, 1, y verdaderamente lo hace. El modelo ha acertado un total de 44 veces. Sin embargo, el modelo ha fallado un total de 19 veces.
  - Cuando predecimos que no sobrevive, 0, y verdaderamente no lo hace. El modelo ha acertado un total de 105 veces. Sin embargo, el modelo ha fallado un total de 15 veces.

In [ ]:
predictions.show(7)

+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex| Age|SibSp|Parch|    Fare|Embarked|            features|     scaled_features|       rawPrediction|         probability|prediction|
+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|       0|     1|  1|27.0|    0|    2|   211.5|     1.0|[1.0,1.0,27.0,0.0...|[0.0,1.0,0.334003...|[-0.4085223322723...|[0.39926649102204...|       1.0|
|       0|     1|  1|28.0|    0|    0| 27.7208|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[-0.1477295871094...|[0.46313462477245...|       1.0|
|       0|     1|  1|28.0|    0|    0| 30.6958|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[-0.1539467996456...|[0.46158913038311...|       1.0|
|       0|     1|  1|28.0|    0|    0|    42.4|     0.0|[1.0,1.0,28.0,0.0...|(7,[1,2,5],

In [ ]:
print(f'Con el modelo de regresión lineal hemos obtenido una precisión de {accuracy_lr:.4f}')

Con el modelo de regresión lineal hemos obtenido una precisión de 0.8142


### Random Forest.
Random Forest se trata de un modelo de clasificación binaria que utiliza árboles de decisión para alcanzar un único resultado. Este modelo es apto cuando en los árboles individuales no están correlacionados entre si.

Los parámetros indicados son los siguientes:
  - `labelCol`: De tipo *double*, es decir, decimal.
  - `featuresCol`: Este debe de ser un vector con las variables independientes.
  - `numTrees`: De tipo *int*, el número de árboles de decisión que se van a construir en el modelo.

Al igual que en el modelo de Árbol de decisión, la predicción devuelve las mismas columnas.

In [ ]:
rf = RandomForestClassifier(labelCol=depent_cols,
                            featuresCol="scaled_features",
                            numTrees=30)
rfModel = rf.fit(train_data)

In [ ]:
predictions = rfModel.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol=depent_cols, predictionCol="prediction", metricName="accuracy")
accuracy_rf = evaluator.evaluate(predictions)

In [ ]:
predictions.show(7)

+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex| Age|SibSp|Parch|    Fare|Embarked|            features|     scaled_features|       rawPrediction|         probability|prediction|
+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|       0|     1|  1|27.0|    0|    2|   211.5|     1.0|[1.0,1.0,27.0,0.0...|[0.0,1.0,0.334003...|[19.1711686708752...|[0.63903895569584...|       0.0|
|       0|     1|  1|28.0|    0|    0| 27.7208|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[20.5668873348942...|[0.68556291116314...|       0.0|
|       0|     1|  1|28.0|    0|    0| 30.6958|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[20.5668873348942...|[0.68556291116314...|       0.0|
|       0|     1|  1|28.0|    0|    0|    42.4|     0.0|[1.0,1.0,28.0,0.0...|(7,[1,2,5],

In [ ]:
predictions.select("prediction", depent_cols).groupBy("prediction", depent_cols).count().show()

+----------+--------+-----+
|prediction|Survived|count|
+----------+--------+-----+
|       1.0|       0|    6|
|       0.0|       0|  114|
|       0.0|       1|   20|
|       1.0|       1|   43|
+----------+--------+-----+



In [ ]:
print(f'Con el modelo de Random Forest hemos obtenido una precisión de {accuracy_rf:.4f}')

Con el modelo de Random Forest hemos obtenido una precisión de 0.8579


### Support Vector Machine
Support Vector Machine son algoritmos de aprendizaje supervisado para la clasificación datos en espacios multidimensionales. Su objetivo es encontrar el hiperplano óptimo con la mejor separación de las clases de datos, maximizando el margen entre ellas. Es efectivo cuando se tiene un dataset de numeroso datos y además es resistente a datos atípicos.


Los parámetros indicados son los siguientes:
  - `labelCol`: De tipo *double*, es decir, decimal.
  - `featuresCol`: Este debe de ser un vector con las variables independientes.
  - `maxIter`: De tipo *int*, cuántas veces el algoritmo intentará ajustar los coeficientes del hiperplano para optimizar la separación entre las clases en el conjunto de entrenamiento.

Al igual que en el modelo de Árbol de decisión, la predicción devuelve las mismas columnas.

In [ ]:
lsvc = LinearSVC(labelCol=depent_cols,
                 featuresCol="scaled_features",
                 maxIter=30)
lsvcModel = lsvc.fit(train_data)

In [ ]:
predictions = lsvcModel.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol=depent_cols, predictionCol="prediction", metricName="accuracy")
accuracy_lsvc = evaluator.evaluate(predictions)

In [ ]:
predictions.select("prediction", depent_cols).groupBy("prediction", depent_cols).count().show()

+----------+--------+-----+
|prediction|Survived|count|
+----------+--------+-----+
|       1.0|       0|   16|
|       0.0|       0|  104|
|       0.0|       1|   22|
|       1.0|       1|   41|
+----------+--------+-----+



In [ ]:
predictions.show(7)

+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex| Age|SibSp|Parch|    Fare|Embarked|            features|     scaled_features|       rawPrediction|prediction|
+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+----------+
|       0|     1|  1|27.0|    0|    2|   211.5|     1.0|[1.0,1.0,27.0,0.0...|[0.0,1.0,0.334003...|[0.99787184222027...|       0.0|
|       0|     1|  1|28.0|    0|    0| 27.7208|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[0.99856352082172...|       0.0|
|       0|     1|  1|28.0|    0|    0| 30.6958|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[0.99855004459767...|       0.0|
|       0|     1|  1|28.0|    0|    0|    42.4|     0.0|[1.0,1.0,28.0,0.0...|(7,[1,2,5],[1.0,0...|[0.99883819091679...|       0.0|
|       0|     1|  1|28.0|    0|    0|221.7792|     0.0|[1.0,1.0,28.0,0.0...|(7,[1,

In [ ]:
print(f'Con el modelo de Support Vector Machine hemos obtenido una precisión de {accuracy_lsvc:.4f}')

Con el modelo de Support Vector Machine hemos obtenido una precisión de 0.7923


### Naive Bayes.

Se trata de un modelo basado en el teorema de Bayes. Se asume que las variables son independientes. Calcula la probabilidad de que una caso pertenezca a una clase dadas una serie de características.


Los parámetros indicados son los siguientes:
  - `labelCol`: De tipo *double*, es decir, decimal.
  - `featuresCol`: Este debe de ser un vector con las variables independientes.

Al igual que en el modelo de Árbol de decisión, la predicción devuelve las mismas columnas.

In [ ]:
nb = NaiveBayes(labelCol=depent_cols,
                featuresCol="scaled_features")
nbModel = nb.fit(train_data)

In [ ]:
predictions = nbModel.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol=depent_cols, predictionCol="prediction", metricName="accuracy")
accuracy_nb = evaluator.evaluate(predictions)

In [ ]:
predictions.select("prediction", depent_cols).groupBy("prediction", depent_cols).count().show()

+----------+--------+-----+
|prediction|Survived|count|
+----------+--------+-----+
|       1.0|       0|    3|
|       0.0|       0|  117|
|       0.0|       1|   45|
|       1.0|       1|   18|
+----------+--------+-----+



In [ ]:
predictions.show(7)

+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex| Age|SibSp|Parch|    Fare|Embarked|            features|     scaled_features|       rawPrediction|         probability|prediction|
+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|       0|     1|  1|27.0|    0|    2|   211.5|     1.0|[1.0,1.0,27.0,0.0...|[0.0,1.0,0.334003...|[-6.3062196892930...|[0.47933566795027...|       1.0|
|       0|     1|  1|28.0|    0|    0| 27.7208|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[-3.6868693337013...|[0.63281632871513...|       0.0|
|       0|     1|  1|28.0|    0|    0| 30.6958|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[-3.7098043292697...|[0.63131321827854...|       0.0|
|       0|     1|  1|28.0|    0|    0|    42.4|     0.0|[1.0,1.0,28.0,0.0...|(7,[1,2,5],

In [ ]:
print(f'Con el modelo de Naive Bayes hemos obtenido una precisión de {accuracy_nb:.4f}')

Con el modelo de Naive Bayes hemos obtenido una precisión de 0.7377


### Preceptrón Multicapa.
Se trata de un tipo de modelo de aprendizaje que consiste en múltiples capas de neuronas interconectadas. Estas redes están organizadas en capas, con una capa de entrada, una o más capas ocultas y una capa de salida. Cada neurona en una capa está conectada a todas las neuronas de la capa siguiente. Además cada una de estas en una capa oculta de las salidas de la capa anterior, aplica una función de activación y pasa el resultado a la siguiente capa.

Los parámetros indicados son los siguientes:
  - `labelCol`: De tipo *double*, es decir, decimal.
  - `featuresCol`: Este debe de ser un vector con las variables independientes.
  - `layers`: Contiene una lista con la capa de entrada, las capas ocultas, y por último la capa de salida. En la capa de entrada debe de haber tantas neuronas como variables independientes; y en la capa de salida debe de haber tantas neuronas como valores únicos tenga la variable objetivo (si es modelo de clasificación).
  - `maxIter`: De tipo *int*, número máximo de iteraciones permitidas.
  - `blockSize`: Cantidad de datos que se procesan en paralelo durante cada iteración del algoritmo de optimización
  - `seed`: Semilla para la generación de números aleatorios, lo que asegura que cada vez que se ejecute este código, la división sea consistente.
  - `solver`: Método utilizado para resolver el problema de optimización durante el entrenamiento del modelo.

Al igual que en el modelo de Árbol de decisión, la predicción devuelve las mismas columnas.




In [ ]:
mlpc=MultilayerPerceptronClassifier(featuresCol="scaled_features",labelCol=depent_cols ,layers = [len(feature_cols), 16, 2], maxIter=1000,blockSize=8,seed=7,solver="gd")

In [ ]:
ann = mlpc.fit(train_data)

In [ ]:
predictions = ann.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol=depent_cols, predictionCol='prediction',metricName='accuracy')
accuracy_mlpc = evaluator.evaluate(predictions)

In [ ]:
predictions.select("prediction", depent_cols).groupBy("prediction", depent_cols).count().show()

+----------+--------+-----+
|prediction|Survived|count|
+----------+--------+-----+
|       1.0|       0|   14|
|       0.0|       0|  106|
|       0.0|       1|   22|
|       1.0|       1|   41|
+----------+--------+-----+



In [ ]:
predictions.show(7)

+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex| Age|SibSp|Parch|    Fare|Embarked|            features|     scaled_features|       rawPrediction|         probability|prediction|
+--------+------+---+----+-----+-----+--------+--------+--------------------+--------------------+--------------------+--------------------+----------+
|       0|     1|  1|27.0|    0|    2|   211.5|     1.0|[1.0,1.0,27.0,0.0...|[0.0,1.0,0.334003...|[-0.2840109410847...|[0.62461807648434...|       0.0|
|       0|     1|  1|28.0|    0|    0| 27.7208|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[-0.0819992482266...|[0.67201241589184...|       0.0|
|       0|     1|  1|28.0|    0|    0| 30.6958|     1.0|[1.0,1.0,28.0,0.0...|[0.0,1.0,0.346569...|[-0.0825100780899...|[0.67175318958980...|       0.0|
|       0|     1|  1|28.0|    0|    0|    42.4|     0.0|[1.0,1.0,28.0,0.0...|(7,[1,2,5],

In [ ]:
print(f'Con el modelo de perceptrón multicapa hemos obtenido una precisión de {accuracy_mlpc:.4f}')

Con el modelo de perceptrón multicapa hemos obtenido una precisión de 0.8033


## Evaluación de todos los modelos.
Tras la realización de todos los modelos de clasificación debemos comparar sus precisiones y ver cuál es la mejor de todas ellas.

In [ ]:
accuracy = [
    ("Regresión Logística", accuracy_lr),
    ("Árbol de decisión", accuracy_dt),
    ("Árbol de decisión: Boosting del gradiente", accuracy_gbt),
    ("Random Forest", accuracy_rf),
    ("Support Vector Machine", accuracy_lsvc),
    ("Naive Bayes", accuracy_nb),
    ("Preceptrón Multicapa", accuracy_mlpc),

]

schema = T.StructType([
    T.StructField("Modelo", T.StringType(), True),
    T.StructField("Accuracy", T.DoubleType(), True),
])


acc_models = spark.createDataFrame(accuracy, schema)

In [ ]:
acc_models.show()

+--------------------+------------------+
|              Modelo|          Accuracy|
+--------------------+------------------+
| Regresión Logística|0.8142076502732241|
|   Árbol de decisión|0.8524590163934426|
|Árbol de decisión...|0.8688524590163934|
|       Random Forest|0.8579234972677595|
|Support Vector Ma...|0.7923497267759563|
|         Naive Bayes|0.7377049180327869|
|Preceptrón Multicapa|0.8032786885245902|
+--------------------+------------------+



In [ ]:
acc_models = acc_models.orderBy(acc_models["Accuracy"].desc()) #orden descendente

In [ ]:
acc_models.show()

+--------------------+------------------+
|              Modelo|          Accuracy|
+--------------------+------------------+
|Árbol de decisión...|0.8688524590163934|
|       Random Forest|0.8579234972677595|
|   Árbol de decisión|0.8524590163934426|
| Regresión Logística|0.8142076502732241|
|Preceptrón Multicapa|0.8032786885245902|
|Support Vector Ma...|0.7923497267759563|
|         Naive Bayes|0.7377049180327869|
+--------------------+------------------+



In [ ]:
best_model = acc_models.first()["Modelo"]
best_acc = acc_models.first()["Accuracy"]
print(f'El mejor modelo para realizar un modelo de clasificación binario es \033[1m{best_model}\033[0m con una precisión de {best_acc:.2f}.')

El mejor modelo para realizar un modelo de clasificación binario es Árbol de decisión: Boosting del gradiente con una precisión de 0.87.


## Cerrar sesión del entorno.

In [ ]:
spark.stop()